In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
X = tf.placeholder(tf.float32,[None,28,28,1])
Y = tf.placeholder(tf.float32,[None,10])
#keep_prob=tf.placeholder(tf.float32)
is_training=tf.placeholder(tf.bool)

In [3]:
#W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))
#L1 = tf.nn.relu(tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME'))
#L1 = tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

L1 = tf.layers.conv2d(X,32,[3,3],activation=tf.nn.relu,padding='SAME')
L1 = tf.layers.max_pooling2d(L1,[2,2],[2,2],padding='SAME')
L1 = tf.layers.dropout(L1,0.7,is_training)

#W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))
#L2 = tf.nn.relu(tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME'))
#L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

L2 = tf.layers.conv2d(L1,64,[3,3],activation=tf.nn.relu)
L2 = tf.layers.max_pooling2d(L2,[2,2],[2,2],padding='SAME')
L2 = tf.layers.dropout(L2,0.7,is_training)

#W3 = tf.Variable(tf.random_normal([7*7*64,256],stddev=0.01))
#L3 = tf.reshape(L2,[-1,7*7*64])
#L3 = tf.nn.relu(tf.matmul(L3,W3))
#L3 = tf.nn.dropout(L3,keep_prob)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3,256,activation=tf.nn.relu)
L3 = tf.layers.dropout(L3,0.7,is_training)

#W4 = tf.Variable(tf.random_normal([256,10],stddev=0.01))
#L4 = tf.matmul(L3,W4)
L4 = tf.layers.dense(L3,10,activation=None)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits=L4))
optimizer=tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [0]:
batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [7]:
for epoch in range(15):
  total_cost = 0
  for i in range(total_batch):
    batch_xs,batch_ys = mnist.train.next_batch(batch_size)
    batch_xs = batch_xs.reshape(-1,28,28,1)
    _,cost_val = sess.run([optimizer,cost],feed_dict={X:batch_xs,Y:batch_ys,is_training:True})
    total_cost += cost_val
  print("Epoch",'%02d'%epoch,'{:.8f}'.format(total_cost/total_batch))
print("Finish")

Epoch 00 0.16931430
Epoch 01 0.04595813
Epoch 02 0.03191339
Epoch 03 0.02107490
Epoch 04 0.01647901
Epoch 05 0.01144917
Epoch 06 0.01019366
Epoch 07 0.00855305
Epoch 08 0.00887384
Epoch 09 0.00559576
Epoch 10 0.00319671
Epoch 11 0.00437755
Epoch 12 0.00586419
Epoch 13 0.00523801
Epoch 14 0.00316183
Finish


In [8]:
accu = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y,1),tf.argmax(L4,1)),tf.float32))
print(sess.run(accu*100,feed_dict={X:mnist.test.images.reshape(-1,28,28,1),Y:mnist.test.labels,is_training:1}))

99.19
